Paso 1. Entender el problema
En este caso se nos pide analizar la implementción de un código que presenta la opción al usuario de definir la presición con la cual se desea analizar la ecuación diferencial parcial para una placa con voltaje definido en una de sus aristas. En este caso el usuario define una diferencia de potencial la cual desea ser la presición de la solución y el programa entonces iterará hasta cumplir con la condición estipulada o hasta cumplir con la condición máxima de parada. Notamos detalles elementales para entonces poder comentar el código en los espacios de la plantilla expuestos. Por ejemplo notamos que se utiliza el metodo de Gauss-Seidel para la solución del ejercicio, no posee carga adicional entonces la solución se simplifica. El número de puntos de la malla es definido de forma constante, en este caso 30 y permite al usuario variar el tamaño de la placa, el voltaje en la arista de la misma y contiene una condición de parada de 500 iteraciones en el cálculo de EDP. Las bibliotecas científicas utilizadas son las comunes de encontrar para este tipo de escenarios, es decir Numpy para soluciones matemáticas y Matplotlib para la graficación de resultados.

Paso 2. Configuración del plan
Una vez entendido el problema se debe leer el código dado por el profesor, notar que cuenta con dos funciones auxiliares que cuentan con herramientas para poder ejecutar la función principal donde el usaurio podrá ingresar parámetros, de igual forma existen definiciones globales en el código como el número de puntos en la malla que no es iterado. De esta forma se puede luego entender el código, poder comentar en los puntos expuestos para finalmente poder realizar el ultimo paso de Poyla donde se criticará e intentará el cambio al código.

Paso 3. Ejecución del plan
Se comenta el código.

In [1]:
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt

def AproxUXY(uxy, pm, prec):
    '''Calcula el valor aproximado del potencial eléctrico en el punto (x, y)

    Parámetros de la función
    ------------------------
    uxy : matriz con los valores iniciales del potencial electrico en cada
           punto de la malla
    pm : número de puntos de la malla
    prec : precisión requerida para el cálculo aproximado de los valores del
            potencial en la malla

    Salida de la función
    --------------------
    valorAproxUXY : matriz con los valores finales del potencial eléctrico
                    cada punto de la malla
    '''

    # Se inicia el contador de iteraciones una variable que permitirá almacenar el número de iteraciones hasta alcanzar la presición deseada
    contador_iteraciones = 0

    # La canasta de impresición se define como un valor arbitrario mayor a 0 para poder entrar en el ciclo de while
    canasta_imprec = 1

    # Se realiza el cálculo iterativo del potencial aproximado en cada punto
    # de la malla iniciando en la esquina superior izquierda de esta, hasta
    # que se alcanza la precisión requerida
    while canasta_imprec > 0:
        # Aumenta el número de iteraciones cada vez que inicia un nuevo ciclo
        contador_iteraciones += 1

        # Se define de esta forma, en caso de que no sea cambiada más adelante entonces esta condición será suficiente para salir del while
        canasta_imprec = 0

        for m in range(1, pm-1, 1):
            for n in range(1, pm-1, 1):
                # Se guarda la solución de la iteración anterior para poder compararla bajo terminos de la condición
                uxy_anterior = uxy[m, n]

                # Se aplica el método de Gauss-Siedel para esta EDP particular
                uxy[m, n] = 0.25*(uxy[m+1,n]+uxy[m-1,n]+uxy[m,n+1]+uxy[m,n-1])

                # Se comparan las diferencias entre dos soluciones iterativas para comparar contra la presición
                dif = np.abs([uxy[m,n]-uxy_anterior])[0]

                # En caso de no alcanzar la presición deseada se cambian las condiciones para volver entrar al ciclo while
                if dif > prec:
                    canasta_imprec += 1

                # Utilizada las siguientes líneas de código como condición de salida al pasar 500 iteraciones para evitar bucles del while
                if contador_iteraciones > 500:
                    canasta_imprec = 0


    valorAproxUXY = uxy
    print("Cantidad iteraciones para alcanzar precisión: ", contador_iteraciones)
    return valorAproxUXY

def Grafico_interactivo(lado, potencial, prec):
    '''Calcula el valor aproximado del potencial eléctrico en una malla de puntos
    y lo

    Parámetros de la función
    ------------------------
    lado : arista de la placa cuadrada sujeta al potencial eléctrico
    potencial : valor del potencial eléctrico en el lado superior de la placa
    prec : precisión requerida para el cálculo aproximado de los valores del
              potencial en la malla

    Salida de la función
    --------------------
    Z : conjunto de valores del potencial aproximado en los puntos de la malla
        y gráfico 3D
    '''

    # Se define la malla de puntos para evaluar el potencial eléctrico
    puntosmalla = 30
    x = np.linspace(0, lado, puntosmalla)
    y = np.linspace(0, lado, puntosmalla)
    X, Y = np.meshgrid(x, y)

    # Se inicializa la matriz con los valores iniciaes del potencial en cada
    # punto de la malla
    uxyi = np.zeros((puntosmalla, puntosmalla), float)

    # Se establecen las condiciones de frontera para el caso particular de la
    # placa cuadrada
    for i in range(0, puntosmalla):
        uxyi[puntosmalla-1,i] = potencial

    # Se calcula el valor aproximado del potencial eléctrico en los puntos de
    # la malla y se asignan al eje Z
    Z = AproxUXY(uxyi, puntosmalla, prec/1000)

    plt.figure(figsize=(10,6))
    ax = plt.axes(projection='3d')
    ax.set_xlabel('x (m)')
    ax.set_ylabel('y (m)')
    ax.set_zlabel('U (V)')
    ax.plot_surface(X, Y, Z, rstride=1, cstride=1,
                    cmap='cividis', edgecolor='none')
    ax.set_title('Aproximacion potencial electrico en la placa')
    plt.show()

    return

y=interactive(Grafico_interactivo, {'manual': True}, lado=widgets.IntSlider(min=1.0, max=5.0, step=1.0, value=1.0, description='L (m):'),
              potencial=widgets.IntSlider(min=25.0, max=150.0, step=25.0, value=25.0, description='U0 (V):'),
              prec=widgets.IntSlider(min=1.0, max=500.0, step=1.0, value=500.0, description='Prec (mV):'))
display(y)

interactive(children=(IntSlider(value=1, description='L (m):', max=5, min=1), IntSlider(value=25, description=…

Paso 4. Mirar hacia atras
Una vez implementado el código y utilizandolo para conseguir valores deseados, obsevando el número de iteraciones, etc. Notamos que este funciona de una excelente manera, la solución se nos da una gráfica deseada prácticamente igual a la analítica. Nunca se llega al número máximo de iteraciones de 500 para la salida forzada del while. Notamos que es posible hacer la aproximacion UXY de la función aproxUXY como una función recusrsiva con condición de salida forzada que funcionaría para una mejor lectura. El criterio de error puede ser cambiado para flexibilizar el error. Se puede seleccionar cierta tolerancia de que el error puede sobrepasarse, por ejemplo el error máximo puede llegar a ser el error máximo + 10% del error o incluso una selección aleatoria de puntos a comparar con los que se analizará el error.

In [2]:
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt

def AproxUXY(uxy, pm, prec):
    '''Calcula el valor aproximado del potencial eléctrico en el punto (x, y)

    Parámetros de la función
    ------------------------
    uxy : matriz con los valores iniciales del potencial electrico en cada
           punto de la malla
    pm : número de puntos de la malla
    prec : precisión requerida para el cálculo aproximado de los valores del
            potencial en la malla

    Salida de la función
    --------------------
    valorAproxUXY : matriz con los valores finales del potencial eléctrico
                    cada punto de la malla
    '''

    # Se inicia el contador de iteraciones una variable que permitirá almacenar el número de iteraciones hasta alcanzar la presición deseada
    contador_iteraciones = 0

    # La canasta de impresición se define como un valor arbitrario mayor a 0 para poder entrar en el ciclo de while
    canasta_imprec = 1

    # Se realiza el cálculo iterativo del potencial aproximado en cada punto
    # de la malla iniciando en la esquina superior izquierda de esta, hasta
    # que se alcanza la precisión requerida
    while canasta_imprec > 0:
        # Aumenta el número de iteraciones cada vez que inicia un nuevo ciclo
        contador_iteraciones += 1

        # Se define de esta forma, en caso de que no sea cambiada más adelante entonces esta condición será suficiente para salir del while
        canasta_imprec = 0

        for m in range(1, pm-1, 1):
            for n in range(1, pm-1, 1):
                # Se guarda la solución de la iteración anterior para poder compararla bajo terminos de la condición
                uxy_anterior = uxy[m, n]

                # Se aplica el método de Gauss-Siedel para esta EDP particular
                uxy[m, n] = 0.25*(uxy[m+1,n]+uxy[m-1,n]+uxy[m,n+1]+uxy[m,n-1])

                # Se comparan las diferencias entre dos soluciones iterativas para comparar contra la presición
                dif = np.abs([uxy[m,n]-uxy_anterior])[0]

                # En caso de no alcanzar la presición deseada se cambian las condiciones para volver entrar al ciclo while
                if dif > prec + 0.1 * prec:
                    canasta_imprec += 1

                # Utilizada las siguientes líneas de código como condición de salida al pasar 500 iteraciones para evitar bucles del while
                if contador_iteraciones > 500:
                    canasta_imprec = 0


    valorAproxUXY = uxy
    print("Cantidad iteraciones para alcanzar precisión: ", contador_iteraciones)
    return valorAproxUXY

def Grafico_interactivo(lado, potencial, prec):
    '''Calcula el valor aproximado del potencial eléctrico en una malla de puntos
    y lo

    Parámetros de la función
    ------------------------
    lado : arista de la placa cuadrada sujeta al potencial eléctrico
    potencial : valor del potencial eléctrico en el lado superior de la placa
    prec : precisión requerida para el cálculo aproximado de los valores del
              potencial en la malla

    Salida de la función
    --------------------
    Z : conjunto de valores del potencial aproximado en los puntos de la malla
        y gráfico 3D
    '''

    # Se define la malla de puntos para evaluar el potencial eléctrico
    puntosmalla = 30
    x = np.linspace(0, lado, puntosmalla)
    y = np.linspace(0, lado, puntosmalla)
    X, Y = np.meshgrid(x, y)

    # Se inicializa la matriz con los valores iniciaes del potencial en cada
    # punto de la malla
    uxyi = np.zeros((puntosmalla, puntosmalla), float)

    # Se establecen las condiciones de frontera para el caso particular de la
    # placa cuadrada
    for i in range(0, puntosmalla):
        uxyi[puntosmalla-1,i] = potencial

    # Se calcula el valor aproximado del potencial eléctrico en los puntos de
    # la malla y se asignan al eje Z
    Z = AproxUXY(uxyi, puntosmalla, prec/1000)

    plt.figure(figsize=(10,6))
    ax = plt.axes(projection='3d')
    ax.set_xlabel('x (m)')
    ax.set_ylabel('y (m)')
    ax.set_zlabel('U (V)')
    ax.plot_surface(X, Y, Z, rstride=1, cstride=1,
                    cmap='cividis', edgecolor='none')
    ax.set_title('Aproximacion potencial electrico en la placa')
    plt.show()

    return

y=interactive(Grafico_interactivo, {'manual': True}, lado=widgets.IntSlider(min=1.0, max=5.0, step=1.0, value=1.0, description='L (m):'),
              potencial=widgets.IntSlider(min=25.0, max=150.0, step=25.0, value=25.0, description='U0 (V):'),
              prec=widgets.IntSlider(min=1.0, max=500.0, step=1.0, value=500.0, description='Prec (mV):'))
display(y)

interactive(children=(IntSlider(value=1, description='L (m):', max=5, min=1), IntSlider(value=25, description=…

In [3]:
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display

import numpy as np
import random
import matplotlib.pyplot as plt


def AproxUXY(uxy, pm, prec):
    '''Calcula el valor aproximado del potencial eléctrico en el punto (x, y)

    Parámetros de la función
    ------------------------
    uxy : matriz con los valores iniciales del potencial electrico en cada
           punto de la malla
    pm : número de puntos de la malla
    prec : precisión requerida para el cálculo aproximado de los valores del
            potencial en la malla

    Salida de la función
    --------------------
    valorAproxUXY : matriz con los valores finales del potencial eléctrico
                    cada punto de la malla
    '''

    uxy_anterior = uxy

    # Se inicia el contador de iteraciones una variable que permitirá almacenar el número de iteraciones hasta alcanzar la presición deseada
    contador_iteraciones = 0

    # La canasta de impresición se define como un valor arbitrario mayor a 0 para poder entrar en el ciclo de while
    canasta_imprec = 1

    # Se realiza el cálculo iterativo del potencial aproximado en cada punto
    # de la malla iniciando en la esquina superior izquierda de esta, hasta
    # que se alcanza la precisión requerida
    while canasta_imprec > 0:
        # Aumenta el número de iteraciones cada vez que inicia un nuevo ciclo
        contador_iteraciones += 1

        # Se define de esta forma, en caso de que no sea cambiada más adelante entonces esta condición será suficiente para salir del while
        canasta_imprec = 0

        for i in range(5):
            pruebax  = random(0, pm)
            pruebay = random(0, pm)
            dif = np.abs([uxy[m, n] - uxy_anterior])[0]

            if dif > prec + 0.1 * prec:
                canasta_imprec = 1

        for m in range(1, pm - 1, 1):
            for n in range(1, pm - 1, 1):
                # Se guarda la solución de la iteración anterior para poder compararla bajo terminos de la condición
                uxy_anterior = uxy[m, n]

                # Se aplica el método de Gauss-Siedel para esta EDP particular
                uxy[m, n] = 0.25 * (uxy[m + 1, n] + uxy[m - 1, n] + uxy[m, n + 1] + uxy[m, n - 1])

                # Se comparan las diferencias entre dos soluciones iterativas para comparar contra la presición
                dif = np.abs([uxy[m, n] - uxy_anterior])[0]

                # En caso de no alcanzar la presición deseada se cambian las condiciones para volver entrar al ciclo while
                if dif > prec + 0.1 * prec:
                    canasta_imprec += 1

                # Utilizada las siguientes líneas de código como condición de salida al pasar 500 iteraciones para evitar bucles del while
                if contador_iteraciones > 500:
                    canasta_imprec = 0

    valorAproxUXY = uxy
    print("Cantidad iteraciones para alcanzar precisión: ", contador_iteraciones)
    return valorAproxUXY


def Grafico_interactivo(lado, potencial, prec):
    '''Calcula el valor aproximado del potencial eléctrico en una malla de puntos
    y lo

    Parámetros de la función
    ------------------------
    lado : arista de la placa cuadrada sujeta al potencial eléctrico
    potencial : valor del potencial eléctrico en el lado superior de la placa
    prec : precisión requerida para el cálculo aproximado de los valores del
              potencial en la malla

    Salida de la función
    --------------------
    Z : conjunto de valores del potencial aproximado en los puntos de la malla
        y gráfico 3D
    '''

    # Se define la malla de puntos para evaluar el potencial eléctrico
    puntosmalla = 30
    x = np.linspace(0, lado, puntosmalla)
    y = np.linspace(0, lado, puntosmalla)
    X, Y = np.meshgrid(x, y)

    # Se inicializa la matriz con los valores iniciaes del potencial en cada
    # punto de la malla
    uxyi = np.zeros((puntosmalla, puntosmalla), float)

    # Se establecen las condiciones de frontera para el caso particular de la
    # placa cuadrada
    for i in range(0, puntosmalla):
        uxyi[puntosmalla - 1, i] = potencial

    # Se calcula el valor aproximado del potencial eléctrico en los puntos de
    # la malla y se asignan al eje Z
    Z = AproxUXY(uxyi, puntosmalla, prec / 1000)

    plt.figure(figsize=(10, 6))
    ax = plt.axes(projection='3d')
    ax.set_xlabel('x (m)')
    ax.set_ylabel('y (m)')
    ax.set_zlabel('U (V)')
    ax.plot_surface(X, Y, Z, rstride=1, cstride=1,
                    cmap='cividis', edgecolor='none')
    ax.set_title('Aproximacion potencial electrico en la placa')
    plt.show()

    return


y = interactive(Grafico_interactivo, {'manual': True},
                lado=widgets.IntSlider(min=1.0, max=5.0, step=1.0, value=1.0, description='L (m):'),
                potencial=widgets.IntSlider(min=25.0, max=150.0, step=25.0, value=25.0, description='U0 (V):'),
                prec=widgets.IntSlider(min=1.0, max=500.0, step=1.0, value=500.0, description='Prec (mV):'))
display(y)

interactive(children=(IntSlider(value=1, description='L (m):', max=5, min=1), IntSlider(value=25, description=…